In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

import yaml
import os

from pprint import pprint

In [ ]:
os.environ['OPENAI_API_KEY'] = yaml.safe_load(open('../credentials.yml'))['openai']

In [ ]:
embedding_function = OpenAIEmbeddings(
    model='text-embedding-ada-002',
)

vectorstore = Chroma(
    persist_directory="data/chroma_1.db",
    embedding_function=embedding_function
)

retriever = vectorstore.as_retriever()

retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7fc48828f040>)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
model = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0.7,
)

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
# * Baseline
result_baseline = model.invoke("What are the top 3 things needed in a good social media marketing strategy for Facebook (Meta)? Site any sources used.")

pprint(result_baseline.content)

('A successful social media marketing strategy for Facebook (Meta) involves '
 'several key components. Here are the top three elements:\n'
 '\n'
 '1. **Target Audience Understanding**: \n'
 '   - Knowing your audience is crucial for effective engagement. This '
 'involves researching demographics, interests, behaviors, and pain points. '
 "Facebook's Audience Insights tool can provide valuable data about your "
 'followers and potential customers. Tailoring content to meet the needs and '
 'preferences of your audience can significantly enhance engagement and '
 'conversion rates. \n'
 '\n'
 '   **Source**: Facebook Business, "Target Your Audience": [Facebook '
 'Business](https://www.facebook.com/business/help/164749007013531)\n'
 '\n'
 '2. **Content Strategy**: \n'
 '   - A diverse content strategy that includes a mix of posts—such as images, '
 'videos, stories, and live streams—will keep your audience engaged. '
 'High-quality visuals and compelling storytelling are essential. Reg

In [ ]:
# * RAG
result = rag_chain.invoke("What are the top 3 things needed in a good social media marketing strategy for Facebook (Meta)? Site any sources used.")

pprint(result)

('Based on the provided context, the top three things needed in a good social '
 'media marketing strategy for Facebook (Meta) include:\n'
 '\n'
 "1. **Clear Objectives**: It's essential to have defined objectives such as "
 'lead generation, lead nurturing, and lead conversion. Every piece of content '
 'should aim to achieve one or more of these objectives to avoid creating '
 'content for the sake of content. (Source: Marley Jaxx)\n'
 '\n'
 '2. **Understanding Your Audience**: Identifying and clarifying who your '
 'ideal customer is (often referred to as an ideal customer avatar) is '
 'crucial. This involves understanding their demographics, geographic details, '
 'and psychographics. By knowing your audience better than they know '
 'themselves, you can tailor your marketing efforts effectively. (Source: Adam '
 'Erhart)\n'
 '\n'
 '3. **Quality Content**: While quality content is important, the emphasis '
 'should also be on frequency and consistency. Engaging with your audience 